In [61]:
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.palettes import OrRd
from bokeh.plotting import figure
from bokeh.transform import transform
from bokeh.models import ColumnDataSource, LinearColorMapper, Range1d, SingleIntervalTicker
from datetime import datetime
import numpy as np

output_notebook()

Loading BokehJS ...

In [133]:
def plotcal(
    df, 
    date_column,
    color_column,
    hover_tooltips=None,
    text_color='grey',
    text_font='courier',
    palette=OrRd[9],
    weekdays=None,
    mode='calendar',
    fig_args={
        'plot_width':300,
        'plot_height':200,
        'toolbar_location':None,
        'tools':'hover',
        'x_axis_location':"above"
    },
    rect_args={
        'width':1,
        'height':1,
        'line_color':'white',
        'line_width':0,
    },

    show_dates=True

    ):
    
    x='day'
    y='week'
    day_of_month_column='dom'
    
    df['date_str'] = df['date'].dt.strftime('%a %b %d, %Y')
    df['month'] = df['date'].dt.strftime('%b')
    df[x] = df['date'].dt.strftime('%a')
    df[day_of_month_column] = df['date'].dt.day
    df[y] = df['date'].dt.isocalendar().week
    
    
    source = ColumnDataSource(df)
    mapper = LinearColorMapper(palette=palette, low=df[color_column].max(), high=df[color_column].min())

    if mode == 'calendar':
        if not weekdays:
            weekdays=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
            
        range_args ={
            'x_range':weekdays,
            'y_range':Range1d(float(df[y].max())+0.5, float(df[y].min())-0.5)#list(reversed([i(i) for i in df[y].unique()]))
        }
        xy = {'x':x, 'y':y}

    elif mode == 'github':
        if not weekdays:
            weekdays=list(reversed(['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']))

        range_args ={
            'x_range':Range1d(float(df[y].min())-0.5, float(df[y].max())+0.5), #Range1d(-1,53),  #[i(i) for i in df[y].unique()],
            'y_range':weekdays
        }
        xy = {'x':y, 'y':x}
    
    p = figure(**{**fig_args, **range_args})

    rect_renderer = p.rect(
        **{**rect_args, **xy}, 
        source=source, 
        fill_color=transform(color_column, mapper)
    )

    if show_dates:
        text_renderer = p.text(
            **xy, 
            text=day_of_month_column,
            text_align='center',
            text_baseline='middle',
            text_color=text_color,
            text_font=text_font,
            source=source
        )

    if range_args['x_range'] != weekdays:
        p.xaxis.ticker = SingleIntervalTicker(interval=1)
        #p.xaxis.major_label_text_font_size = '10px'

    if range_args['y_range'] != weekdays:
        p.yaxis.ticker = SingleIntervalTicker(interval=1)
        #p.yaxis.major_label_text_font_size = '10pt'

    labels = {int(v):k for k,v in dict(df.groupby(['month'])['week'].min()).items()}
    
    for i in df['week'].unique():
        if i not in labels.keys():
            labels[int(i)] = ''
            
    p.axis.major_label_overrides = labels
    
    
    p.axis.major_label_text_color=text_color
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.outline_line_color = None
    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.minor_tick_line_color = None
    p.axis.major_label_standoff = 0
    p.hover.tooltips = hover_tooltips
    rect_renderer.nonselection_glyph.fill_alpha=1
    
    return p, source

# Generate data

In [134]:
df = pd.DataFrame(
    {'date': pd.date_range('2020-01-01','2020-12-31')}
)
df['val'] = np.random.rand(len(df))

# df['date_str'] = df['date'].dt.strftime('%a %b %d %y')
# df['month'] = df['date'].dt.strftime('%b')
# df['day'] = df['date'].dt.strftime('%a')
# df['dom'] = df['date'].dt.day
# df['week'] = df['date'].dt.isocalendar().week

# df = df.set_index('date')

# Calendar plots

In [135]:
p, _ = plotcal(
    df, 
    date_column='date', 
    color_column='val',
    mode='github', 
    fig_args={
        'plot_width':900,
        'plot_height':175,
        'tools':'hover',
        'toolbar_location':None,
        'x_axis_location':"above"
    }, 
    hover_tooltips=[
        ('Value','@val'),
        ('Date','@date_str')
    ], 
    show_dates=False
)
show(p)

In [136]:
p, _ = plotcal(
    df.loc[(
        df['date'] > (datetime.today() - pd.Timedelta('31 days'))
    ) & (df['date'] < datetime.today())
    ].copy(), 
    
    date_column='date', 
    color_column='val',
    mode='calendar',   
    fig_args={
        'plot_width':400,
        'plot_height':200,
        'tools':'hover',
        'toolbar_location':None,
        'x_axis_location':"above"
    }, 
    hover_tooltips=[
        ('Value','@val'),
        ('Date','@date_str')
    ], 
    show_dates=True
)

show(p)